# 代码要求：使用五倍交叉，训练100轮，使用早停技术

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

# test

In [ ]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms
from torch.autograd import Variable

In [ ]:
import torch
import numpy as np
import skimage
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(1)  # reproducible
torch.set_default_tensor_type(torch.DoubleTensor)
from PIL import Image
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
])

In [ ]:
# -*- coding: UTF-8 -*-
"""
@FileName: MobileNetV1.py
@Description: Implement MobileNetV1
@Author: Ryuk
@CreateDate: 2019/12/9 13:35
@LastEditTime: 2019/12/9 13:35
@LastEditors: Please set LastEditors
@Version: v1.0
"""

import time
import torch
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable


class MobileNet(nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True))

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),

                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True))

        self.feature = nn.Sequential(
            conv_bn(1, 32, 2),
            conv_dw(32, 64, 1),
            conv_dw(64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AvgPool2d(7) )

        self.fc = nn.Linear(1024, 8)
        self.logsoftmax = nn.LogSoftmax(dim=1)
    def forward(self, x):
        x = self.feature(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return self.logsoftmax(x)



def main():
    x = torch.randn(32, 1, 224, 224)
    net = MobileNet()
    print(net(x))


if __name__ == '__main__':
    main()

tensor([[-2.2058, -2.1214, -2.1306, -2.3809, -2.2629, -1.9754, -2.0439, -1.6795],
        [-2.1892, -2.0654, -2.0936, -2.4782, -2.1737, -1.9401, -2.0993, -1.7481],
        [-2.2729, -2.0918, -2.0801, -2.4192, -2.1380, -1.9708, -2.1069, -1.7106],
        [-2.2124, -2.1238, -2.0239, -2.5300, -2.1695, -1.9616, -2.0067, -1.7726],
        [-2.1401, -2.0832, -2.1495, -2.5172, -2.2200, -1.8659, -2.0799, -1.7586],
        [-2.1962, -2.1255, -2.0896, -2.5006, -2.1425, -1.8851, -2.0999, -1.7603],
        [-2.1087, -2.0685, -2.1195, -2.4481, -2.2855, -1.9910, -2.0723, -1.7065],
        [-2.1999, -2.0028, -2.1338, -2.3778, -2.1556, -2.0565, -2.1379, -1.7068],
        [-2.2501, -2.0171, -2.0931, -2.4526, -2.1304, -1.9699, -2.1309, -1.7418],
        [-2.1908, -2.0183, -2.1281, -2.4875, -2.2104, -2.0093, -2.0694, -1.6976],
        [-2.2652, -2.0057, -2.1764, -2.4577, -2.1135, -1.8766, -2.2022, -1.7256],
        [-2.2383, -2.0783, -2.1641, -2.4182, -2.0951, -1.9994, -2.0666, -1.7193],
        [-2.3554

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = MobileNet().to(device)

In [ ]:
loss_fn = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.006,momentum=0.5)

In [ ]:
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold as kFold

In [ ]:
x_train=np.load("/content/drive/MyDrive/summer_start/data_generate/train_1channel.npy")
y_train=np.load("/content/drive/MyDrive/summer_start/data_generate/label.npy")

In [ ]:
kfold =kFold(n_splits=5,shuffle=True,random_state=1)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data,label):
        self.data = data #加载npy数据
        self.label = label
        self.transforms = transform #转为tensor形式
    def __getitem__(self, index):
        hdct= self.data[index, :, :, :]  # 读取每一个npy的数据
        hdct = np.squeeze(hdct)  # 删掉一维的数据，就是把通道数这个维度删除
#         ldct = 2.5 * skimage.util.random_noise(hdct * (0.4 / 255), mode='poisson', seed=None) * 255 #加poisson噪声
#         hdct=Image.fromarray(np.uint8(hdct)) #转成image的形式
#         ldct=Image.fromarray(np.uint8(ldct)) #转成image的形式
        hdct= self.transforms(hdct)  #转为tensor形式
#         ldct= self.transforms(ldct)  #转为tensor形式
        return hdct, self.label[index] #返回数据还有标签
    def __len__(self):
        return self.data.shape[0] #返回数据的总个数

In [14]:
labels=[0,1,2,3,4,5,6,7]
list_score=[]
for fold, (train_index, test_index) in enumerate(kfold.split(x_train, y_train)):
    print("train_index",train_index)
    print("test_index",test_index)
    ### Dividing data into folds
    x_train_fold = x_train[train_index]
    x_test_fold = x_train[test_index]
    y_train_fold = y_train[train_index]
    y_test_fold = y_train[test_index]
    train = MyDataset(x_train_fold, y_train_fold)
    test = MyDataset(x_test_fold, y_test_fold)
    train_loader = DataLoader(train, batch_size = 8, shuffle = True)
    test_loader = DataLoader(test, batch_size = 8, shuffle = False)
    print("fold:",fold)
    max_score=[]
    # model = Net().to(device)
    best_acc=0
    model = MobileNet().to(device)
    loss_fn = nn.NLLLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.5)
    for i in range(50):
      list1=[]
      for t, (data, target) in enumerate(train_loader):
        # print("t",t)
        data,target = Variable(data.to(device)),Variable(target.to(device))
        # print(data.shape)
        pred = model(data.double().to(device))
        loss = loss_fn(pred,target)
        
        list1.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print("loss",loss.item())
      print("netloss",i,np.mean(list1))
      with torch.no_grad():
        list_loss=[]
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data.double().to(device))
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            pred_loss=np.exp(output.data.cpu())
            logsloss=log_loss(target.cpu().detach().numpy(),pred_loss,labels=labels)
            list_loss.append(logsloss)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        print('accuracy:{:.3f}%'.format(
            100. * correct / len(test_loader.dataset)))
        val_acc=correct / len(test_loader.dataset)
        print("log_loss",sum(list_loss)/len(list_loss))
        print("*"*20)
        if val_acc > best_acc:
            best_acc = val_acc
            es = 0
        else:
            es += 1
            print("Counter {} of 5".format(es))
            if es > 5:
                print("Early stopping with best_acc: ", best_acc, "and val_acc for this epoch: ", val_acc, "...")
                break
    list_score.append(best_acc)
    print("*"*40)
from numpy import mean
print(mean(list_score))


train_index [    0     1     2 ... 13884 13885 13886]
test_index [    4     5     6 ... 13861 13872 13877]
fold: 0
netloss 0 1.468732850299196
accuracy:51.440%
log_loss 1.3203302751089807
********************
netloss 1 1.2428574934638008
accuracy:59.719%
log_loss 1.116474240850368
********************
netloss 2 1.0567160545476408
accuracy:67.999%
log_loss 0.9627563849430674
********************
netloss 3 0.8781487578625958
accuracy:74.406%
log_loss 0.7719035307396569
********************
netloss 4 0.7363873021263176
accuracy:77.502%
log_loss 0.6842610121748496
********************
netloss 5 0.6561851600198888
accuracy:80.850%
log_loss 0.6289280486346678
********************
netloss 6 0.5958634488487454
accuracy:80.706%
log_loss 0.6095235365110425
********************
Counter 1 of 5
netloss 7 0.5468204302034793
accuracy:82.325%
log_loss 0.5542596504455639
********************
netloss 8 0.5141552404505431
accuracy:82.937%
log_loss 0.5369335177470775
********************
netloss 9 0.46713

In [ ]:
print(mean(list_score))
